In [2]:
import torch
import pandas as pd
import pickle
import numpy as np

In [3]:
val = pd.read_pickle("../data/ml-1m-split/val.pkl")
train = pd.read_pickle("../data/ml-1m-split/train.pkl")

In [4]:
freqs = train.groupby("user").size().reset_index().rename(columns={0: "num_items"})

In [5]:
freqs

,user,num_items
0,0,37
1,1,90
2,2,36
3,3,15
4,4,139
...,...,...
6035,6035,622
6036,6036,141
6037,6037,14
6038,6038,86


In [6]:
num_items_col = []
prev_user = 0
prev_num_items = freqs.iloc[prev_user].num_items
for row in val.itertuples(index=False):
  if row.user != prev_user:
    prev_user = row.user
    prev_num_items = freqs.iloc[prev_user].num_items
    if prev_user % 10000 == 0:
      print(prev_user)
  num_items_col.append(prev_num_items)
  

In [7]:
len(num_items_col)

149914

In [8]:
val["user_num_items"] = num_items_col

In [9]:
group = val.groupby("user_num_items")

In [10]:
val

,user,item,item_id,rating,timestamp,user_num_items
2,0,853,914,3,978301968,37
11,0,877,938,4,978301752,37
18,0,2889,3105,5,978301713,37
19,0,2592,2797,4,978302039,37
35,0,740,783,4,978824291,37
...,...,...,...,...,...,...
1000173,6039,2534,2739,2,956715942,239
1000180,6039,971,1036,3,956715455,239
1000191,6039,3508,3751,4,964828782,239
1000197,6039,1840,2020,3,956715288,239


In [12]:
i = 0
j = 0
batch_size = 200
batches = []
for (user_num_items, block) in group:
  i += 1
  if j > 900:
    batch_size = 100
  #print(row.sample(frac=1))
  for (idx, batch) in block.sample(frac=1).groupby(np.arange(block.shape[0]) // batch_size):
    batches.append(batch)
    j += 1
  if i % 10 == 0:
    print(i)
    
print(j)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
1119


In [ ]:
row0 = group.nth(0)

In [ ]:
row0

In [ ]:
for (i, r) in therow.groupby(np.arange(therow.shape[0]) // 32):
  print(r)
  if i == 5:
    break

In [71]:
len(batches)

831

In [57]:
with open("../data/ml-20m-split/val_batched.pkl", "wb") as f:
  pickle.dump(batches, f)

In [28]:
arr = np.array(batches[52342].drop(["item_id", "timestamp", "user_num_items"], axis=1))

In [30]:
arr.dtype

dtype('float64')

In [22]:
batches2 = np.stack([np.array(batches[i].drop(["item_id", "timestamp", "user_num_items"], axis=1)) for i in range(len(batches))])

KeyboardInterrupt: 

In [13]:
to_stack = []
for i in range(len(batches)):
  if i % 10000 == 0:
    print(i)
  to_stack.append(np.array(batches[i].drop(["item_id", "timestamp", "user_num_items"], axis=1)))

0


In [24]:
batches2 = np.stack(to_stack)

ValueError: all input arrays must have the same shape

In [14]:
np.save("../data/ml-1m-split/val_batched_h_1m.npy", to_stack)

In [33]:
batches[160]

,user,item,item_id,rating,timestamp,user_num_items
961914,5794,1614,1779,1,958145902,191
725635,4343,1124,1214,5,965233258,191
726118,4343,2533,2738,4,965242019,191
961273,5794,3268,3501,5,958147749,191
725179,4343,2286,2479,1,966273528,191
...,...,...,...,...,...,...
961177,5794,1958,2139,5,958146036,191
960984,5794,1634,1806,1,958147180,191
726089,4343,2512,2717,2,965330221,191
961711,5794,423,437,1,958149220,191


In [15]:
len(to_stack)

1119